In [ ]:
from sqlalchemy import create_engine, inspect, select, text, and_, or_, func
from sqlalchemy import Table, MetaData

import pandas as pd

In [ ]:
w_engine = create_engine('mysql+pymysql://e0376929:' + 'wsUQu4z0' + '@localhost:3306/olist', echo=True)

In [ ]:
def order_category():
    query = 'select n.english, count(*) from order_items as o inner join products as p on(o.product_id = p.product_id) inner join prod_cat_name as n on (p.category_name = n.portugese) group by n.english order by count(*) desc limit 10;'
    with w_engine.connect() as con:
        query_re_ca = con.execute(query)
    re_category = query_re_ca.fetchall()
    t = []
    # t
    for i in range(len(re_category)):
        # tt = re_category[i][0]
        t.append(re_category[i][0][:-1])
    return t

In [ ]:
tt = order_category()
tt

In [ ]:
def product_rank()
    # get result from SQL
    q = """select o.product_id, avg(score) as avg_rank, avg(price) as avg_price, sum(order_item_id) as total_sale
    from order_items as o left join order_reviews as o_re using(order_id) 
    left join products as p on(o.product_id = p.product_id)
    group by o.product_id;"""
    with w_engine.connect() as con:
        d1 = con.execute(q)
    s1 = d1.fetchall()
    s1 = pd.DataFrame(s1)
    
    # rename the dataframe
    s1.rename(columns={0: "product_id", 1: "avg_score", 2: "avg_price", 3: "total_sale"}, errors="raise", inplace = True)
    
    # processing the result
    s1[["avg_score","avg_price", "total_sale"]] = s1[["avg_score","avg_price", "total_sale"]].astype('float')
    s1[["avg_price", "total_sale"]] = np.log10(s1[["avg_price", "total_sale"]])
    scaler = preprocessing.MinMaxScaler()
    s1[["avg_score","avg_price", "total_sale"]] = scaler.fit_transform(s1[["avg_score","avg_price", "total_sale"]])
    
    #generate the total scoreing
    s1["total ranking"] = 0.7* s1["total_sale"] -0.2* s1["avg_price"] + 0.1 * s1["avg_score"]
    s1.sort_values(by = ["total ranking"],ascending=False, inplace = True)
    
    return s1["product"][:10]

In [ ]:
def rank_price(category):
    q = """select o.product_id, seller_id, avg(price)
    from order_items AS o
            INNER JOIN
        products AS p ON (o.product_id = p.product_id)
            INNER JOIN
        prod_cat_name AS n ON (p.category_name = n.portugese)
    where n.english like :search_key
    group by product_id, seller_id
    order by avg(price)
    limit 10"""
    
    category = category + "%"
    with w_engine.connect() as con:
        rs = con.execute(text(q).bindparams(search_key = category))

    result = rs.fetchall()
    return result